In [4]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
from create_dataframe import create_dataframe

In [12]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', None)

In [6]:
create_dataframe()

df = pd.read_csv('grants.csv')
df

,project_name,grant_name,year,organization,organization_short_name,lat,lon
0,Дистанционные радиофизические предвестники засух в стратегически важных аграрных регионах России (на примере Алтайского края),Проведение фундаментальных научных исследований и поисковых научных исследований отдельными научными группами,2022.0,Федеральное государственное бюджетное учреждение науки Институт водных и экологических проблем Сибирского отделения Российской академии наук,Институт водных и экологических проблем Сибирского отделения Российской академии наук,54.840900,83.112100
1,Создание и исследование элементов системы организации цепочки поставок древесного сырья с использованием технологии блокчейн и интеллектуального анализа данных,Проведение фундаментальных научных исследований и поисковых научных исследований отдельными научными группами,2022.0,"федеральное государственное автономное образовательное учреждение высшего образования ""Северный (Арктический) федеральный университет имени М. В. Ломоносова""",Северный (Арктический) федеральный университет имени М. В. Ломоносова,64.529000,40.549000
2,Изучение структурных особенностей и структурных превращений лигнинов при различных способах делигнификации,Проведение фундаментальных научных исследований и поисковых научных исследований отдельными научными группами,2022.0,"федеральное государственное автономное образовательное учреждение высшего образования ""Северный (Арктический) федеральный университет имени М. В. Ломоносова""",Северный (Арктический) федеральный университет имени М. В. Ломоносова,64.529000,40.549000
3,"Биологически активные лигнаны в растительном сырье и отходах лесной промышленности: идентификация, определение, выделение",Проведение фундаментальных научных исследований и поисковых научных исследований отдельными научными группами,2022.0,"федеральное государственное автономное образовательное учреждение высшего образования ""Северный (Арктический) федеральный университет имени М. В. Ломоносова""",Северный (Арктический) федеральный университет имени М. В. Ломоносова,64.529000,40.549000
4,Оценка современного состояния пресноводных экосистем Соловецкого архипелага (фундаментальный и прикладной аспекты),Проведение фундаментальных научных исследований и поисковых научных исследований отдельными научными группами,2022.0,Федеральное государственное бюджетное учреждение науки Федеральный исследовательский центр комплексного изучения Арктики имени академика Н.П. Лаверова Уральского отделения Российской академии наук,Федеральный исследовательский центр комплексного изучения Арктики имени академика Н.П. Лаверова Уральского отделения Российской академии наук,NaN,NaN
...,...,...,...,...,...,...,...
8753,"Разработка материалов и создание теоретических основ получения новых полимерных композиционных материалов на основе термоэластопластов, обеспечивающих эффективное гашение вибраций природного и техногенного происхождения в широком температурно-частотном диапазоне, в том числе в условия Арктики и Крайнего Севера РФ",Проведение инициативных исследований молодыми учеными,2021.0,"Федеральное государственное бюджетное образовательное учреждение высшего образования ""Вятский государственный университет""",Вятский государственный университет,58.602239,49.665994
8754,Технологии контролируемого синтеза гибридных светопоглощающих наноматериалов методами лазерной абляции в жидкости для высокоэффективной фототермической конверсии солнечного излучения,Проведение инициативных исследований молодыми учеными,2021.0,"Федеральное государственное автономное образовательное учреждение высшего образования ""Дальневосточный федеральный университет""",Дальневосточный федеральный университет,43.028640,131.894495
8755,Исследование влияния случайных перегрузок на околопороговые характеристики роста усталостной трещины в металлах,Проведение инициативных исследований молодыми учеными,2021.0,"Федеральное государственное бюджетное образовательное учреждение высшего образования ""Волгоградский государственный технический 

In [7]:
px.set_mapbox_access_token(open(".mapbox_token").read()[:-1])

In [8]:
organizations = df \
    .groupby(['organization_short_name', 'year'], as_index=False, dropna=False) \
    .agg({'project_name': 'count', 'lat': 'max', 'lon': 'max'}) \
    .rename(columns={"project_name" : 'number_of_projects'})

In [9]:
years = df.year.sort_values().unique()

fig = px.scatter_mapbox(organizations[organizations.year==years[0]],
                        lat='lat',
                        lon='lon',
                        hover_name="organization_short_name",
                        size='number_of_projects',
                        center=dict(lat=60, lon=50),
                        zoom=3)



app = JupyterDash(__name__)


app.layout = html.Div([
    html.P("Select a year:"),
    dcc.RadioItems(
        id='year', 
        options=years,
        value=years[0],
        inline=True
    ),
    dcc.Graph(figure=fig, id='graph'),
])

@app.callback(
    Output("graph", "figure"), 
    Input("year", "value"))
def display_scatter_mapbox(year):
    if year:
        organization_by_year = organizations[organizations.year==year]
    else:
        organization_by_year = organizations[organizations.year.isna()]
    fig = px.scatter_mapbox(organization_by_year,
                        lat='lat',
                        lon='lon',
                        hover_name="organization_short_name",
                        size='number_of_projects',
                        center=dict(lat=60, lon=50),
                        zoom=3)
    return fig



app.run_server(mode='inline', port=8090, dev_tools_ui=True,
              dev_tools_hot_reload=True, threaded=True)

Dash is running on http://127.0.0.1:8090/



[INFO] Dash is running on http://127.0.0.1:8090/



In [10]:
years = df.year.sort_values().unique()

fig = px.density_mapbox(organizations[organizations.year==years[0]], 
                        lat='lat', 
                        lon='lon', 
                        z='number_of_projects', 
                        radius=10,
                        center=dict(lat=60, lon=50), 
                        zoom=2)



app = JupyterDash(__name__)


app.layout = html.Div([
    dcc.RadioItems(
        id='year', 
        options=years,
        value=years[0],
        inline=True
    ),
    dcc.Graph(figure=fig, id='graph'),
])

@app.callback(
    Output("graph", "figure"), 
    Input("year", "value"))
def display_density_mapbox(year):
    if year:
        organization_by_year = organizations[organizations.year==year]
    else:
        organization_by_year = organizations[organizations.year.isna()]
    fig = px.density_mapbox(organization_by_year, 
                        lat='lat', 
                        lon='lon', 
                        z='number_of_projects', 
                        radius=10,
                        center=dict(lat=60, lon=50), 
                        zoom=2)
    return fig



app.run_server(mode='inline', port=8091, dev_tools_ui=True,
              dev_tools_hot_reload=True, threaded=True)

Dash is running on http://127.0.0.1:8091/



[INFO] Dash is running on http://127.0.0.1:8091/



In [13]:
without_location = df[df.lat.isna()] \
    .groupby('organization_short_name', as_index=False) \
    .agg({'project_name': 'count'}) \
    .sort_values(by='project_name', ascending=False) \
    .reset_index(drop=True)
without_location

,organization_short_name,project_name
0,"Федеральный исследовательский центр ""Институт общей физики им. А.М. Прохорова Российской академии наук""",54
1,Институт органической химии им. Н.Д. Зелинского Российской академии наук,48
2,Физико-технический институт им. А.Ф. Иоффе Российской академии наук,46
3,"Федеральный исследовательский центр ""Пущинский научный центр биологических исследований Российской академии наук""",44
4,Институт проблем экологии и эволюции им. А.Н. Северцова Российской академии наук,37
5,Институт химии растворов им. Г.А. Крестова Российской академии наук,25
6,Федеральный исследовательский центр комплексного изучения Арктики имени академика Н.П. Лаверова Уральского отделения Российской академии наук,23
7,"""приволжский исследовательский медицинский университет"" министерства здравоохранения российской федерации",14
8,Ордена Трудового Красного Знамени Институт солнечно-земной физики Сибирского отделения Российской академии наук,14
9,"Федеральный исследовательский центр ""Саратовский научный центр Российской академии наук""",9
